In [50]:
import pandas as pd


pv = pd.read_csv('./processed_data/pv/DORM_2021_PV_no_interpol.csv', index_col = 0)
net = pd.read_csv('./processed_data/netload/DORM_2021_netload.csv', index_col=0 )
net = net.round(1)
load_index_x = pd.read_csv('./processed_data/load/X_load_231days.csv', index_col=0).index
load_index_y = pd.read_csv('./processed_data/load/Y_load_231days.csv', index_col=0).index
load_index = set(load_index_x) | set(load_index_y)




In [51]:
display(pv)
display(net)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
210101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18.8,6.8,12.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
210102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,...,33.1,28.5,18.9,4.9,0.0,0.0,0.0,0.0,0.0,0.0
210103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,8.4,...,23.0,5.9,2.6,0.7,0.0,0.0,0.0,0.0,0.0,0.0
210104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.3,...,31.8,7.2,3.8,1.3,0.0,0.0,0.0,0.0,0.0,0.0
210105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,4.4,...,15.1,7.0,5.4,1.4,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1,11.2,...,15.9,3.2,2.2,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
211228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,4.3,...,5.6,3.9,1.4,0.1,0.0,0.0,0.0,0.0,0.0,0.0
211229,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,3.9,...,4.0,3.1,12.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
211230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,10.4,...,27.0,4.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
210101,149.8,123.6,116.8,124.8,126.7,113.4,115.2,112.0,122.9,118.2,...,127.9,133.8,131.4,137.1,126.3,134.1,134.0,130.1,131.6,138.9
210102,132.5,123.8,121.2,127.6,119.7,105.4,103.5,107.9,101.0,120.2,...,123.4,120.9,125.0,137.5,136.6,138.4,148.4,141.3,130.4,124.4
210103,129.0,123.4,129.7,117.0,116.8,109.9,104.2,116.8,105.9,114.7,...,117.3,126.5,128.3,137.4,159.6,138.8,147.4,140.9,150.1,147.9
210104,139.0,130.4,115.5,101.3,119.0,97.5,107.4,101.0,135.6,140.2,...,108.7,100.5,113.2,123.7,149.4,148.3,146.3,157.3,157.9,150.3
210105,138.9,126.5,117.0,108.7,117.7,104.3,114.0,99.9,128.2,130.8,...,138.9,121.8,119.8,143.5,155.9,146.4,150.8,154.4,154.6,167.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211226,215.2,190.8,182.4,176.6,170.7,170.8,155.8,162.3,170.5,161.2,...,168.4,173.5,171.9,191.3,190.1,186.6,197.6,209.3,213.8,211.4
211227,191.2,193.3,184.2,181.0,195.0,187.5,193.3,178.3,189.7,202.0,...,143.2,181.6,168.2,169.2,188.2,177.0,183.8,186.6,197.3,190.0
211228,188.9,185.9,180.4,164.4,163.4,174.1,145.8,158.3,156.0,145.0,...,140.3,159.2,155.4,156.4,164.4,166.1,154.8,175.9,167.9,179.6
211229,165.0,161.3,163.1,160.6,153.3,151.4,150.5,149.1,146.3,147.3,...,144.7,137.6,145.3,135.2,192.8,176.2,190.4,166.6,166.2,167.8


In [86]:
# select only the data in load_index from pv
pv = pv.loc[load_index]
net = net.loc[load_index]
load = pd.DataFrame(index=list(load_index), columns=range(24))

for i in range(len(load_index)):
    load.iloc[i] = pv.iloc[i] + net.iloc[i]

C:\Users\user\AppData\Local\Temp\ipykernel_16220\1314572931.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  pv = pv.loc[load_index]
C:\Users\user\AppData\Local\Temp\ipykernel_16220\1314572931.py:3: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  net = net.loc[load_index]


In [91]:
type(load)

pandas.core.frame.DataFrame

In [98]:
# check if load has nan value or not
load.isnull().values.any()


False

In [96]:
load.to_csv("./processed_data/load/DORM_2021_load.csv")